Session storage of Google Colab is used.

In [ ]:
# !pip install pillow

from PIL import Image

# Function to hide a message in an image
def hide_message(image_path, message, output_image_path):
    image = Image.open(image_path)
    pixels = list(image.getdata()) # pixel data is converted into a list of pixels

    # Converting the message to ASCII 8 bit binary format
    binary_message = ''.join(format(ord(char), '08b') for char in message)

    # Since images have limited capacity to hide data and to preserve the image quality
    # I have added a line which checks if the length of the binary message is greater than three times
    # the number of pixels in the image. If it is, it raises an error because the message is too long to be hidden in the image
    if len(binary_message) > len(pixels) * 3:
        raise ValueError("The message is too long!")

    # To indicate the end of the message, a sentinal (16 bits- unusual to occur) are added
    binary_message += '1111111111111110'

    data_index = 0
    for i in range(len(pixels)):
        pixel = list(pixels[i])
        for j in range(3):
            if data_index < len(binary_message):
                pixel[j] = int(bin(pixel[j])[:-1] + binary_message[data_index], 2)
                data_index += 1
        pixels[i] = tuple(pixel)

    stego_image = Image.new(image.mode, image.size)
    stego_image.putdata(pixels)
    stego_image.save(output_image_path)

    # Display the steganographic image
    stego_image.show()

# Function to extract a message from an image
def extract_message(image_path):
    image = Image.open(image_path)
    pixels = list(image.getdata())

    binary_message = ''
    for pixel in pixels:
        for color_channel in pixel:
            binary_message += bin(color_channel)[-1]

    sentinel_index = binary_message.find('1111111111111110')
    if sentinel_index == -1:
        return "No hidden message found."

    binary_message = binary_message[:sentinel_index]
    message = ''
    for i in range(0, len(binary_message), 8):
        message += chr(int(binary_message[i:i+8], 2))

    return message


hide_message("/content/singleBirdinsky221.jpeg", "Let your dreams take flight like a bird in the sky", "output_image.png")
extracted_message = extract_message("output_image.png")
print("Extracted Message:", extracted_message)


Extracted Message: Let your dreams take flight like a bird in the sky
